# CIS600 - Social Media Data Mining 
###  
<img src="https://www.syracuse.edu/wp-content/themes/g6-carbon/img/syracuse-university-seal.svg?ver=6.3.9" style="width: 200px;"/>

# Twitter

###  October 4, 2018

Today we'll use *Python Twitter Tools* from *sixohsix* to get some data from Twitter. In order to use this notebook, you must install the python package `twitter`

In order to make this work, you will need to store your keys and tokens in a dicionary called "auth_dict"

We are going to need the `twitter` package. You can download the package [here](https://github.com/sixohsix/twitter) and use `easy_install`. But you can also find it in [*PyPi*](https://pypi.org/), i.e. you can install it using `pip`.

To go the latter route, just make sure you conda environment is activated and then go like this:

` pip install twitter `

In [3]:
from twitter import *

In [4]:
import json
with open('auth_dict','r') as f:
    twtr_auth = json.load(f)
    
# To make it more readable, lets store
# the OAuth credentials in strings first.
CONSUMER_KEY = twtr_auth['consumer_key']
CONSUMER_SECRET = twtr_auth['consumer_secret']
OAUTH_TOKEN = twtr_auth['token']
OAUTH_TOKEN_SECRET = twtr_auth['token_secret']
    
# Then, we store the OAuth object in "auth"
auth = OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)
# Notice that there are four tokens - you need to create these in the
# Twitter Apps dashboard after you have created your own "app".

Here we create a `twitter` API wrapper object.

In [6]:
# We now create the twitter search object.
t = Twitter(auth=auth)

But what is it, exactly?

In [ ]:
help(t)

Let's get some *statuses* from my timeline.

In [8]:
my_tmln = t.statuses.home_timeline()

[Here](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html) is the Twitter.com documentation on statuses.

Here I get the timeline of another user, Zed Shaw. This might not work for you if you are not a follower of his.

In [18]:
zeds_tmln = t.statuses.user_timeline(screen_name="zedshaw")

Therefore, here is another account.

In [19]:
ppoly_tmln = t.statuses.user_timeline(screen_name="primalpoly")

In [ ]:
ppoly_tmln[0]

OK, now we'll break it down, starting from a more basic request.

In [10]:
docs_example = """GET https://api.twitter.com/1.1/search/tweets
                .json?q=%23freebandnames&since_id=2401261998405
                1000&max_id=250126199840518145&result_type=mixed&count=4"""

Note that we used triple quotes (i.e. 3 double quotes) for this multiline string. It is still just a string. What is going on in this string?

The first piece of information in the string *docs_example* that stands out is "freebandnames". That's a search term, but what is its effect? It is preceded by "%23" to indicate *hashtags*. See [this](https://brajeshwar.github.io/entities/) for more.

Rmk: [There is no such thing as plaintext.](https://www.joelonsoftware.com/2003/10/08/the-absolute-minimum-every-software-developer-absolutely-positively-must-know-about-unicode-and-character-sets-no-excuses/)

Next, we see "since_id" - we are searching for tweets tweeted since a given tweet ID number.

The "max_id" gives the other bound - we want no tweets with IDs greater than this value.

We are asking for "mixed" results - both popular and recent

Count refers to results per page.

**But we don't want to build such a string every time we do a query, and that is one reason for the API wrapper provided by `twitter`**

In [21]:
frebandnames = t.search.tweets(q='#freebandnames', since_id=24012619984051000,
                result_type='mixed', count=4)

In [ ]:
frebandnames.keys()

In [23]:
freeb = frebandnames['statuses']

In [ ]:
freeb[0]['user']['screen_name']

**More Examples**

In [ ]:
# Get your "home" timeline
t.statuses.home_timeline()

# Get a particular friend's timeline
t.statuses.user_timeline(screen_name="zedshaw")

# to pass in GET/POST parameters, such as `count`
t.statuses.home_timeline(count=5)

# to pass in the GET/POST parameter `id` you need to use `_id`
t.statuses.oembed(_id=1234567890)

# Update your status
t.statuses.update(
    status="Here is another tweet.")

# Send a direct message
#t.direct_messages.new(
#    user="primalpoly",
#    text="Geoffy-baby, big fan of your work.") # Try not to spam him, guys

# Get the members of tamtar's list "Things That Are Rad"
t.lists.members(owner_screen_name="buffer", slug="the-buffer-team")

# An *optional* `_timeout` parameter can also be used for API
# calls which take much more time than normal or twitter stops
# responding for some reason:
#t.users.lookup(
#    screen_name=','.join(A_LIST_OF_100_SCREEN_NAMES), _timeout=1)

# Rmk: A_LIST_OF_100_SCREEN_NAMES is not defined. Why not fix that?

# Overriding Method: GET/POST
# you should not need to use this method as this library properly
# detects whether GET or POST should be used, Nevertheless
# to force a particular method, use `_method`
t.statuses.oembed(_id=1234567890, _method='GET')

In [ ]:
# Search for the latest tweets about #pycon
t.search.tweets(q="#pycon")

# Search for the latest tweets about #pycon, using extended mode
t.search.tweets(q="#pycon", tweet_mode='extended')

We can search trends by geographic region. The Yahoo! Where On Earth ID for the entire world is 1, for example. Look [here](https://developer.twitter.com/en/docs/trends/trends-for-location/api-reference/get-trends-place) for more.

**Streaming**

In [ ]:
# Rmk: this cell will not run!
twitter_stream = TwitterStream(auth=OAuth(...))
iterator = twitter_stream.statuses.sample()

for tweet in iterator:
    #...do something with this tweet...

In [ ]:
# Create a *streaming* connection (not RESTful, different from Search).
t_stream = TwitterStream(auth=auth)


# Get an *iterator* object from the twitter wrapper

tweeterator = t_stream.statuses.sample()


# The loop below simply prints randomly selected new tweets
# until we reach the threshold of "tweet_count"

tweet_count = 100
for tweet in tweeterator:
    tweet_count -= 1
    print(json.dumps(tweet))  
    if tweet_count <= 0:
        break 

In [26]:
WORLD_WOE_ID = 1
US_WOE_ID = 23424977

Let's get the trends (top 50) for the whole world, and then for the US.

In [ ]:
# Prefix ID with the underscore for query string parameterization.
# Without the underscore, the twitter package appends the ID value
# to the URL itself as a special case keyword argument.

world_trends = t.trends.place(_id=WORLD_WOE_ID)
us_trends = t.trends.place(_id=US_WOE_ID)

print(world_trends)
print('\n')
print(us_trends)

That doesn't look so good, so now we "pretty print" it using the JSON module. The builtin *print* gives better output when its input is a JSON object (rather than, say, a string or list object).

In [ ]:
list(us_trends)[0]['trends'][4]['name']

In [ ]:
#print(json.dumps(world_trends, indent=1))
print('\n')
print(json.dumps(us_trends, indent=1))

Now that we have both results in memory, we can combine them. For example, we find the intersection of the top world trends and the top USA trends.

In [ ]:
# Why bother with "set" here?
world_trends_set = set([trend['name'] for trend in world_trends[0]['trends']])

us_trends_set = set([trend['name'] for trend in us_trends[0]['trends']]) 

common_trends = world_trends_set.intersection(us_trends_set) # intersection is a set method

print(common_trends)

Next, let's collect search results in a loop. Note that we are already able to do this with the Search API and that the Streaming API will be needed for larger volume.

The *count* parameter represents the number of tweets we want *per page*.

This is important because the Twitter REST API uses *cursoring* to organize large search results in *pages*. The next example shows how to use the cursor. Read more about it [here](https://developer.twitter.com/en/docs/basics/cursoring)

In [ ]:
# Set the next variable to any trending topic
# or anything else for that matter.
q = '"Super Bowl"' 

# Same as before - the number of tweets we want *per page*.
count = 1000

# We do the search call.
search_results = t.search.tweets(q=q, count=count)

# Remember 'status' refers to the actual content of a tweet.
# (as opposed to the metadata)
statuses = search_results['statuses']


# Iterate through 5 more batches of results by following the cursor.
# The use of the underscore "_" below is a convention in python 
# indicating to the reader that the variable will not be used for
# anything within the loop.
for _ in range(5):
    print("Length of statuses ", len(statuses))
    # Remember "try...except"? Here it is in action:
    try:
        next_results = search_results['search_metadata']['next_results']
    # No more results when next_results doesn't exist
    except KeyError as e: 
        break
        
    # Create a dictionary from next_results, which has the following form:
    # ?max_id=313519052523986943&q=NCAA&include_entities=1
    kwargs = dict([ kv.split('=') for kv in next_results[1:].split("&") ]) 
    
    search_results = t.search.tweets(**kwargs) #Another API call
    statuses += search_results['statuses'] #Appending to statuses results

# Show one sample search result by slicing the list...
print(json.dumps(statuses[0], indent=1))

Note the "except" line in the cell above. Including "as e" after the "KeyError" lets you access the attributes of the error - we didn't do that, but you might find a need for it. Inside the indent, you could manipulate the object e and get its attributes, such as e.args.

Instead, we simply break out of the loop.

In the next example, we extract text, screen names and hashtags from tweets.

In [ ]:
len(statuses)

In [ ]:
status_texts = [ status['text'] for status in statuses ]

screen_names = [ user_mention['screen_name'] for status in statuses
                     for user_mention in status['entities']['user_mentions'] ]

hashtags = [ hashtag['text'] for status in statuses
                 for hashtag in status['entities']['hashtags'] ]

# Compute a collection of all words from all tweets
words = [ w for t in status_texts for w in t.split() ]

# Explore the first 5 items for each...

print(json.dumps(status_texts[0:5], indent=1))
print(json.dumps(screen_names[0:5], indent=1))
print(json.dumps(hashtags[0:5], indent=1))
print(json.dumps(words[0:5], indent=1))

We can create a basic frequency distribution from words in tweets - first glimpse at NLP

In [ ]:
from collections import Counter

# Recall that you can loop through any
# iterator in python - such as a list
# of lists!
for item in [words, screen_names, hashtags]:
    c = Counter(item)
    print(c.most_common()[:10]) # top 10

In [ ]:
# A function for computing lexical diversity
def lexical_diversity(tokens):
    return 1.0*len(set(tokens))/len(tokens) 

# A function for computing the average number of words per tweet
def average_words(statuses):
    total_words = sum([ len(s.split()) for s in statuses ]) 
    return total_words/len(statuses)

print(lexical_diversity(words))
print(lexical_diversity(screen_names))
print(lexical_diversity(hashtags))
print(average_words(status_texts))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# First, we get the frequencies in order.
word_counts = sorted(Counter(words).values(), reverse=True)

# We can plot it along log-scaled axes
plt.loglog(word_counts)
plt.ylabel("Freq")
plt.xlabel("Word Rank")
plt.show()

Here that is in `bokeh`:

In [ ]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

p = figure(title="log axis example", y_axis_type="log", x_axis_type='log',
           y_range=(.9, 400))

p.line(range(len(word_counts)), word_counts, legend="word counts",
       line_color="blue")
show(p)

We can also do histograms.

In [ ]:
for label, data in (('Words', words), 
                    ('Screen Names', screen_names), 
                    ('Hashtags', hashtags)):

    # Build a frequency map for each set of data
    # and plot the values
    c = Counter(data)
    plt.hist(c.values())
    
    # Add a title and y-label ...
    plt.title(label)
    plt.ylabel("Number of items in bin")
    plt.xlabel("Bins (number of times an item appeared)")
    
    # ... and display as a new figure
    plt.show()

**Exercise** Do the above in `bokeh`

And now for something completely different...finding the most popular retweets

In [ ]:
retweets = [
            # We are building a list of tuples
            (status['retweet_count'], 
             status['retweeted_status']['user']['screen_name'],
             status['text'],
             status['retweeted_status']) 
            
            # ... for each status ...
            for status in statuses 
            
            # ... so long as the status meets this condition.
                if 'retweeted_status' in status
           ]

In [ ]:
retweets

Looking up users who have retweeted a 'status'.

First, let's pick one of the retweets. We will find the id of the original tweet from the 'retweeted_status' node (the last element of our tuple).

In [ ]:
retweets[0][-1]['id']

This is the id of the retweeted tweet. We will use this to search Twitter again.

In [ ]:
# Note that we are doing another search here.
rtwtd_id = 1047848885073403904 # Fill this in
_retweets = t.statuses.retweets(id=rtwtd_id)
print([r['user']['screen_name'] for r in _retweets])

In [ ]:
counts = [count for count, _, _, _ in retweets]

plt.hist(counts)
plt.title("Retweets")
plt.xlabel('Bins (number of times retweeted)')
plt.ylabel('Number of tweets in bin')

print(counts)
plt.show()

**Twitter Functions**

We want to reuse the code from the cells above. Also, we want to write clean and readable code when we implement longer and more complex procedures. Here is a function we have already used:

In [24]:
def oauth_login():
    # Replace credentials below with appropriate values - 
    # this works in the TwitterAPIcontd notebook because
    # we defined the credentials above
    CONSUMER_KEY = CONSUMER_KEY
    CONSUMER_SECRET = CONSUMER_SECRET
    OAUTH_TOKEN = OAUTH_TOKEN
    OAUTH_TOKEN_SECRET = OAUTH_TOKEN_SECRET
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
    CONSUMER_KEY, CONSUMER_SECRET)
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

Next, we can wrap the lines that give us trends in a function.

In [25]:
def twitter_trends(twitter_api, woe_id):
    # Prefix ID with the underscore for query string parameterization.
    # Without the underscore, the twitter package appends the ID value
    # to the URL itself as a special-case keyword argument.
    return twitter_api.trends.place(_id=woe_id)

Likewise, we define a function for the looped twitter search.

In [26]:
def twitter_search(twitter_api, q, max_results=200, **kw):
    search_results = twitter_api.search.tweets(q=q, count=100, **kw)
    statuses = search_results['statuses']
    # Enforce a reasonable limit
    max_results = min(1000, max_results)
    for _ in range(10): # 10*100 = 1000
        try:
            next_results = search_results['search_metadata']['next_results']
        except KeyError as e: # No more results when next_results doesn't exist
            break
        # Create a dictionary from next_results, which has the following form:
        # ?max_id=313519052523986943&q=NCAA&include_entities=1
        kwargs = dict([ kv.split('=') 
                       for kv in next_results[1:].split("&") ])
        search_results = twitter_api.search.tweets(**kwargs)
        statuses += search_results['statuses']
        if len(statuses) > max_results:
            break
    
    return statuses

You will likely encounter errors in mining Twitter data. Here is a function to automate the handling of certain errors. See *Mining the Social Web* for more details.

In [61]:
import sys
import time
from twitter.api import TwitterHTTPError
from urllib.error import URLError
from http.client import BadStatusLine

def make_twitter_request(twitter_api_func, max_errors=10, *args, **kw):
    # A nested helper function that handles common HTTPErrors. Return an updated
    # value for wait_period if the problem is a 500 level error. Block until the
    # rate limit is reset if it's a rate limiting issue (429 error). Returns None
    # for 401 and 404 errors, which requires special handling by the caller.
    def handle_twitter_http_error(e, wait_period=2, sleep_when_rate_limited=True):
        if wait_period > 3600: # Seconds
            print('Too many retries. Quitting.', file=sys.stderr)
            raise e
        if e.e.code == 401:
            return None
        elif e.e.code == 404:
            print('Encountered 404 Error (Not Found)', file=sys.stderr)
            return None
        elif e.e.code == 429:
            print('Encountered 429 Error (Rate Limit Exceeded)', file=sys.stderr)
            if sleep_when_rate_limited:
                print("Retrying in 15 minutes...ZzZ...", file=sys.stderr)
                sys.stderr.flush()
                time.sleep(60*15 + 5)
                print('...ZzZ...Awake now and trying again.', file=sys.stderr)
                return 2
            else:
                raise e # Caller must handle the rate limiting issue
        elif e.e.code in (500, 502, 503, 504):
            print('Encountered %i Error. Retrying in %i seconds' % (e.e.code, wait_period), file=sys.stderr)
            time.sleep(wait_period)
            wait_period *= 1.5
            return wait_period
        else:
            raise e

    # End of nested helper function

    wait_period = 2
    error_count = 0
    while True:
        try:
            return twitter_api_func(*args, **kw)
        except TwitterHTTPError as e:
            error_count = 0
            wait_period = handle_twitter_http_error(e, wait_period)
            if wait_period is None:
                return
        except URLError as e:
            error_count += 1
            print("URLError encountered. Continuing.", file=sys.stderr)
            if error_count > max_errors:
                print("Too many consecutive errors...bailing out.", file=sys.stderr)
                raise
        except BadStatusLine as e:
            error_count += 1
            print >> sys.stderr, "BadStatusLine encountered. Continuing."
            if error_count > max_errors:
                print("Too many consecutive errors...bailing out.", file=sys.stderr)
                raise

In [62]:
response = make_twitter_request(t.users.lookup, screen_name="SocialWebMining")

We will want to write responses to disk, on the fly, so that we can collect many observations for later analysis. In *Mining the Social Web*, the Mongo DB database program is recommended. Here is another way (that also can be adapted so that it writes to a database).

First, let's wrap the extraction of tweet "[entities](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/entities-object)" in a function which takes a list of statuses as input.

In [64]:
def extract_tweet_entities(statuses):
    if len(statuses) == 0:
        return [], [], [], [], []
    screen_names = [ user_mention['screen_name'] 
                    for status in statuses
                        for user_mention in status['entities']['user_mentions'] ]
    hashtags = [ hashtag['text']
                    for status in statuses
                        for hashtag in status['entities']['hashtags'] ]
    urls = [ url['expanded_url']
                    for status in statuses
                        for url in status['entities']['urls'] ]
    symbols = [ symbol['text']
                    for status in statuses
                        for symbol in status['entities']['symbols'] ]
    if status['entities'].has_key('media'):
        media = [ media['url']
            for status in statuses
                for media in status['entities']['media'] ]
    else:
        media = []
    return screen_names, hashtags, urls, media, symbols

The above function is a good template of sorts. You can use it to develop your own function for extracting other information from a *list of statuses*. 

Let's define two more functions, then we'll sample from the Twitter stream, writing results as we go. The functions below will take statuses, extract some information, and then write the results into a CSV file using Pandas.


In [65]:
def extract_tweet_basics(status):
    screen_name = None
    tweet_ID = None
    text = None
    if 'user' in status:
        screen_name = status['user']['screen_name'] 
        tweet_ID = status['id']
        text = status['text']
    return screen_name, tweet_ID, text

In [66]:
def tweet_to_csv(file, status):
    screen_name, tweet_ID, text = extract_tweet_basics(status)
    df = pd.DataFrame([[screen_name,tweet_ID,text]], columns=['screen_name','tweet_ID','text'])
    with open(file, 'a') as f:
        df.to_csv(f,header=False, index=False)

Finally, we stick this in the streaming loop from last time.

In [71]:
# Create a *streaming* connection (not RESTful, different from Search).
t_stream = TwitterStream(auth=auth)


# Get an *iterator* object from the twitter wrapper

tweeterator = t_stream.statuses.sample()

# Create a CSV file with column names
# but no data (yet).
import pandas as pd
df = pd.DataFrame(columns=['screen_name','tweet_ID','text'])
df.to_csv('my_csv.csv', index=False)


# The loop below will grab a new tweet,
# extract some basic info, put that info
# in a dataframe object, then use that
# dataframe object to append one row to
# the existing CSV file, 'my_csv.csv'.

tweet_count = 100
for tweet in tweeterator:
    tweet_count -= 1
    tweet_to_csv('my_csv.csv', tweet)  
    if tweet_count <= 0:
        break 

**Network Structure**

Below we'll develop some tools for crawling the friendship graph of some Twitter followers. This exercise is taken directly from *Mining the Social Web*.

In [ ]:
# This will let us create new partial
# functions with arguments set to 
# certain values.
from functools import partial

# This was maxint.
# There is no longer a maxint (in Python 3)
from sys import maxsize


def get_friends_followers_ids(twitter_api, screen_name=None, user_id=None,
                                friends_limit=maxsize, followers_limit=maxsize):
    # Must have either screen_name or user_id (logical xor)
    assert (screen_name != None) != (user_id != None), \
    "Must have screen_name or user_id, but not both"
    
    # You can also do this with a function closure.
    get_friends_ids = partial(make_twitter_request, twitter_api.friends.ids,
                                count=5000)
    get_followers_ids = partial(make_twitter_request, twitter_api.followers.ids,
                                count=5000)
    friends_ids, followers_ids = [], []
    for twitter_api_func, limit, ids, label in [
            [get_friends_ids, friends_limit, friends_ids, "friends"],
            [get_followers_ids, followers_limit, followers_ids, "followers"]
            ]:
        #LOOK HERE! This little line is important.
        if limit == 0: continue
        cursor = -1
        while cursor != 0:
            # Use make_twitter_request via the partially bound callable...
            if screen_name:
                response = twitter_api_func(screen_name=screen_name, cursor=cursor)
            else: # user_id
                response = twitter_api_func(user_id=user_id, cursor=cursor)
            if response is not None:
                ids += response['ids']
                cursor = response['next_cursor']
            print('Fetched {0} total {1} ids for {2}'.format(len(ids),
                    label, (user_id or screen_name), file=sys.stderr))
            if len(ids) >= limit or response is None:
                break
    # Do something useful with the IDs, like store them to disk...
    return friends_ids[:friends_limit], followers_ids[:followers_limit]


In [ ]:
friends_ids, followers_ids = get_friends_followers_ids(t,
                                screen_name="ZedShaw",
                                friends_limit=10,
                                followers_limit=10)
print(friends_ids)
print(followers_ids)

In [ ]:
# Create a mostly empty data frame,
# and write it to a CSV file.
df = pd.DataFrame(columns=['ID','followers'])
df.to_csv('followers.csv', index=False)

# Our function
def save_followers(fid, followers):
    df = pd.DataFrame([[fid, followers]], columns=['ID','followers'])
    with open('followers.csv', 'a') as f:
        df.to_csv(f,header=False, index=False)

In [ ]:
def crawl_followers(twitter_api, screen_name, limit=1000000, depth=2):
    
    # Resolve the ID for screen_name and start working with IDs for consistency
    seed_id = str(twitter_api.users.show(screen_name=screen_name)['id'])
    _, next_queue = get_friends_followers_ids(twitter_api, user_id=seed_id,
                        friends_limit=0, followers_limit=limit)
    
    # Store a seed_id => _follower_ids mapping in MongoDB
    save_followers(seed_id, ','.join([str(x) for x in next_queue]))
    
    d = 1
    # Note that in the example in the next cell,
    # we never enter this loop.
    while d < depth:
        d += 1
        # Reset the next_queue so that we can
        # start building up the next level
        # of followers-of-followers
        (queue, next_queue) = (next_queue, [])
        # Loop through the current
        # level of followers
        for fid in queue:
            _, follower_ids = get_friends_followers_ids(twitter_api, user_id=fid,
                                friends_limit=0, followers_limit=limit)
            # Store an ID with a string recording
            # IDs of followers of the user with ID "fid"
            save_followers(str(fid), ','.join([str(x) for x in follower_ids]))
            # Extending the list
            next_queue += follower_ids
